In [ ]:
!pip install pytest-playwright
!playwright install
!pip install asyncio
!pip install playwright
!playwright install
!playwright install-deps
!pip install playwright_stealth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 MB 12.0 MB/s eta 0:00:00
(node:284) [DEP0169] DeprecationWarning: `url.parse()` behavior is not standardized and prone to errors that have security implications. Use the WHATWG URL API instead. CVEs are not issued for `url.parse()` vulnerabilities.
(Use `node --trace-deprecation ...` to show where the warning was created)
164.7 MiB [] 0% 0.0s164.7 MiB [] 0% 73.8s164.7 MiB [] 0% 38.8s164.7 MiB [] 0% 27.3s164.7 MiB [] 0% 23.3s164.7 MiB [] 0% 15.8s164.7 MiB [] 1% 9.1s164.7 MiB [] 1% 6.9s164.7 MiB [] 2% 6.0s164.7 MiB [] 2% 5.5s164.7 MiB [] 3% 5.2s164.7 MiB [] 3% 5.0s164.7 MiB [] 4% 5.1s164.7 MiB [] 4% 5.2s164.7 MiB [] 5% 4.6s164.7 MiB [] 5% 4.4s164.7 MiB [] 6% 4.2s164.7 MiB [] 6% 4.1s164.7 MiB [] 7% 4.0s164.7 MiB [] 7% 4.1s164.7 MiB [] 8% 4.0s164.7 MiB [] 9% 3.9s164.7 MiB [] 10% 3.8s164.7 MiB [] 10% 3.6s164.7 MiB [] 11% 3.4s164.7 MiB [] 12% 3.2s164.7 MiB [] 13% 3.1s164.7 MiB [] 14% 3.0s164.7 MiB [] 15% 3.0s164.7 MiB [] 15% 2.9s164.7 MiB

In [ ]:
from playwright.sync_api import sync_playwright
import asyncio
import random
from playwright.async_api import async_playwright
from playwright_stealth import Stealth
import os

In [ ]:
async def main():
    async with Stealth().use_async(async_playwright()) as p:
        browser = await p.chromium.launch()

        #Делаем пользователя для входа
        context = await browser.new_context(
            viewport={'width': 1920, 'height': 1080},
            user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/121.0',
            locale='de-DE',
            timezone_id='Europe/Berlin'
        )

        #Переходим на сайт
        page = await context.new_page()
        await page.goto('https://spb.cian.ru/cat.php?deal_type=sale&engine_version=2&from_developer=1&object_type%5B0%5D=2&offer_type=flat&only_flat=1&region=2')

        await page.wait_for_timeout(5000)
        #Фоткаем, чтобы узнать, что получилось
        await page.screenshot(path='cian-results.png')

        #Записываем html с первых 5 страниц
        for i in range(1, 54):
          await page.wait_for_timeout(5000)
          #Считываем код страницы
          html = await page.evaluate("document.documentElement.outerHTML")
          #Записываем
          with open(f'file{i}.html', 'w', encoding='utf-8') as f:
            f.write(html)
          await page.wait_for_timeout(5000)
          await page.screenshot(path=f'cian-results{i}.png')
          #Кликаем на следующую страницу
          await page.get_by_role('link', name=str(i + 1), exact=True).last.click()

        await browser.close()

#Запускаем
await main()

In [ ]:
import os
import shutil

# Создаем архив со всем содержимым текущей директории (так как работали в colab)
!zip -r /content/session_files_sbp.zip /content/
from google.colab import files
files.download('/content/session_files_sbp.zip')

  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/default_configs.db (deflated 98%)
  adding: content/.config/.last_update_check.json (deflated 24%)
  adding: content/.config/active_config (stored 0%)
  adding: content/.config/.last_survey_prompt.yaml (stored 0%)
  adding: content/.config/gce (stored 0%)
  adding: content/.config/config_sentinel (stored 0%)
  adding: content/.config/configurations/ (stored 0%)
  adding: content/.config/configurations/config_default (deflated 15%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2025.12.11/ (stored 0%)
  adding: content/.config/logs/2025.12.11/14.33.40.102128.log (deflated 92%)
  adding: content/.config/logs/2025.12.11/14.34.28.289959.log (deflated 58%)
  adding: content/.config/logs/2025.12.11/14.34.12.303029.log (deflated 58%)
  adding: content/.config/logs/2025.12.11/14.34.39.029902.log (deflated 57%)
  adding: content/.config/logs/2025.12.11/14.34.38.212031.log 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import re

async def parse_with_playwright_selectors(html_content: str):
    async with async_playwright() as p:

        #Здесь не надо делать пользователя искусственно, так как обрабатываем файл
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context()
        page = await context.new_page()
        #Загружаем файл
        await page.set_content(html_content, wait_until='domcontentloaded')
        #Нахожим карточки объектов
        cards = await page.locator('article[data-name="CardComponent"]').all()
        all_data = []

        #Для каждого объекта
        for card in cards:
            data = {}
            #Ищем цену
            price_locator = card.locator('span[data-mark="MainPrice"]')
            if await price_locator.count() > 0:
                data['price'] = await price_locator.first.text_content()
            #Ищем цену за метр
            price_per_m2_locator = card.locator('p[data-mark="PriceInfo"]')
            if await price_per_m2_locator.count() > 0:
                data['price_per_m2'] = await price_per_m2_locator.first.text_content()
            #Ссылка на страницу объекта
            link_locator = card.locator('a[href*="/sale/flat/"]').first
            if await link_locator.count() > 0:
                data['page_url'] = await link_locator.get_attribute('href')
            #Описание
            desc_locator = card.locator('div[data-name="Description"]')
            if await desc_locator.count() > 0:
                data['description'] = await desc_locator.first.text_content()
            #Заголовок
            title_locator = card.locator('span[data-mark="OfferTitle"]')
            if await title_locator.count() > 0:
                data['title'] = await title_locator.first.text_content()
            #Подзаголовок
            subtitle_locator = card.locator('span[data-mark="OfferSubtitle"]')
            if await subtitle_locator.count() > 0:
                data['subtitle'] = await subtitle_locator.first.text_content()
            #Метро
            metro_locator = card.locator('div[data-name="SpecialGeo"]')
            if await metro_locator.count() > 0:
                metro_text = await metro_locator.first.text_content()
                data['metro'] = metro_text

            all_data.append(data)
        await browser.close()

        return pd.DataFrame(all_data)

#Запускаем для всех файлов
async def mainn():
  df_list = []
  for i in range(1, 53):
    with open(f'file{i}.html', 'r', encoding='utf-8') as f:
        html = f.read()
    df = await parse_with_playwright_selectors(html)
    df_list.append(df)
    #Объединяем фреймы
    df_full = pd.concat(df_list, ignore_index=True)
    #Записываем в файл
    df_full.to_csv('cian_parsed.csv', index=False, encoding='utf-8-sig')

#Запускаем
await mainn()

In [ ]:
#Здесь открываем файл заново и скачиваем карточки каждой квартиры по отдельности
df = pd.read_csv('cian_parsed.csv')

async def main_d():
    async with Stealth().use_async(async_playwright()) as p:
        browser = await p.chromium.launch()
        user_agents = [
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/120.0.0.0',
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) Safari/605.1.15',
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/121.0'
        ]
        for i, url in enumerate(df['page_url'].tolist()):
          user_agent_ch = random.choice(user_agents)
          context = await browser.new_context(
          viewport={'width': 1366, 'height': 768},
          locale='ru-RU',
          user_agent=user_agent_ch
          )
          page = await context.new_page()
          await page.goto(url)
          await page.wait_for_timeout(5000)
          #Считываем код страницы
          html = await page.evaluate("document.documentElement.outerHTML")
          #Записываем
          with open(f'file{i}.html', 'w', encoding='utf-8') as f:
            f.write(html)
          await page.wait_for_timeout(5000)
          await context.close()


        await browser.close()

#Запускаем
await main_d()

После скачивания каждой страницы объекта, извлекаем из каждого файла информацию

In [ ]:
import re
import json
import html
import pandas as pd
from typing import Dict, List, Optional, Any
from bs4 import BeautifulSoup

In [ ]:
import re
import json
import html
from typing import Dict, Any
from bs4 import BeautifulSoup
import pandas as pd

def extract_cian_config(html_content):
    # Извлекаем данные из JavaScript-конфигурации window._cianConfig
    config_data = {}

    # Ищем JavaScript с конфигурацией
    script_patterns = [
        r'window\._cianConfig\[\'frontend-offer-card\'\]\s*=\s*(.*?});\s*</script>',
        r'window\._cianConfig\[\'frontend-offer-card\'\]\s*=\s*window\._cianConfig\[\'frontend-offer-card\'\]\s*\|\|\s*\[\]\s*\)\.concat\(\[(.*?)\]\)',
        r'defaultState.*?({.*?})["\']',
        r'offer.*?({.*?})["\']',
    ]

    for pattern in script_patterns:
        matches = re.findall(pattern, html_content, re.DOTALL | re.MULTILINE)
        if matches:
            try:
                # Пытаемся распарсить JSON
                for match in matches:
                    # Очищаем данные
                    cleaned_data = match.strip()
                    if cleaned_data.startswith('[') or cleaned_data.startswith('{'):
                        try:
                            # Декодируем HTML-сущности
                            cleaned_data = html.unescape(cleaned_data)

                            # Удаляем лишние символы
                            cleaned_data = cleaned_data.replace('\\"', '"')
                            cleaned_data = cleaned_data.replace('\\/', '/')
                            cleaned_data = cleaned_data.replace('\\u002F', '/')

                            # Парсим JSON
                            if cleaned_data.startswith('['):
                                data = json.loads(cleaned_data)
                                # Если это массив объектов конфигурации
                                for item in data:
                                    if isinstance(item, dict) and 'key' in item:
                                        config_data[item['key']] = item.get('value', item)
                            elif cleaned_data.startswith('{'):
                                data = json.loads(cleaned_data)
                                config_data.update(data)
                        except json.JSONDecodeError:
                            # Пробуем очистить еще
                            try:
                                # Удаляем лишние escape-символы
                                cleaned_data = cleaned_data.replace('\\"', '"').replace('\\n', ' ').replace('\\t', ' ')
                                data = json.loads(cleaned_data)
                                config_data.update(data)
                            except:
                                continue
            except Exception:
                continue

    return config_data

def extract_offer_data_from_config(config_data):
    # Извлекаем данные предложения из конфигурации
    offer_data = {}

    # Ищем данные предложения в конфигурации
    if 'defaultState' in config_data:
        default_state = config_data['defaultState']
        if isinstance(default_state, dict):
            # Основные данные предложения
            if 'offerData' in default_state and 'offer' in default_state['offerData']:
                offer = default_state['offerData']['offer']

                # Базовые данные
                offer_data['id'] = offer.get('id')
                offer_data['cian_id'] = offer.get('cianId')
                offer_data['object_guid'] = offer.get('objectGuid')
                offer_data['user_id'] = offer.get('userId')
                offer_data['category'] = offer.get('category')

                # Цена и условия сделки
                if 'bargainTerms' in offer:
                    bargain = offer['bargainTerms']
                    offer_data['price'] = {
                        'amount': bargain.get('price'),
                        'currency': bargain.get('currency'),
                        'price_type': bargain.get('priceType'),
                        'prices': bargain.get('prices', {})
                    }

                # Характеристики квартиры
                offer_data['characteristics'] = {
                    'flat_type': offer.get('flatType'),
                    'total_area': offer.get('totalArea'),
                    'living_area': offer.get('livingArea'),
                    'decoration': offer.get('decoration'),
                    'floor_number': offer.get('floorNumber'),
                    'is_apartments': offer.get('isApartments', False)
                }

                # Характеристики здания
                if 'building' in offer:
                    building = offer['building']
                    offer_data['building'] = {
                        'material_type': building.get('materialType'),
                        'floors_count': building.get('floorsCount'),
                        'ceiling_height': building.get('ceilingHeight'),
                        'total_area': building.get('totalArea'),
                        'passenger_lifts_count': building.get('passengerLiftsCount'),
                        'cargo_lifts_count': building.get('cargoLiftsCount'),
                        'deadline': building.get('deadline')
                    }

                # Информация о ЖК
                if 'newbuilding' in offer:
                    nb = offer['newbuilding']
                    offer_data['newbuilding'] = {
                        'id': nb.get('id'),
                        'name': nb.get('name'),
                        'house': nb.get('house'),
                        'developer': nb.get('developer'),
                        'status': nb.get('newbuildingFeatures', {}).get('deadlineInfo')
                    }

    return offer_data

def parse_cian_html(html_content):
    # Парсит HTML-код страницы ЦИАН и извлекает всю информацию
    soup = BeautifulSoup(html_content, 'html.parser')

    result = {
        'basic_info': {},
        'characteristics': {},
        'location': {},
        'metadata': {},
        'additional': {},
        'housing_complex': {},
        'developer_info': {},
        'offer_data': {}
    }

    # Извлекаем данные из JavaScript конфигурации
    config_data = extract_cian_config(html_content)

    # Извлекаем данные предложения из конфигурации
    offer_data_from_config = extract_offer_data_from_config(config_data)
    result['offer_data'] = offer_data_from_config

    # Заголовок
    title_elem = soup.find('h1', class_='xa15a2ab7--_96386--title')
    if title_elem:
        result['basic_info']['title'] = title_elem.get_text(strip=True)

    # Жилой комплекс
    parent_elem = soup.find('div', {'data-name': 'ParentNew'})
    if parent_elem:
        result['basic_info']['residential_complex'] = parent_elem.find('a').get_text(strip=True) if parent_elem.find('a') else None

    # Характеристика
    factoid_items = soup.find_all('div', {'data-name': 'ObjectFactoidsItem'})

    for item in factoid_items:
        label_elem = item.find('span', class_=lambda x: x and 'color_gray60_100' in x)
        value_elem = item.find('span', class_=lambda x: x and 'color_text-primary-default' in x and 'fontWeight_bold' in x)

        if label_elem and value_elem:
            label = label_elem.get_text(strip=True)
            value = value_elem.get_text(strip=True)

            if 'Общая площадь' in label:
                result['characteristics']['total_area'] = value
            elif 'Жилая площадь' in label:
                result['characteristics']['living_area'] = value
            elif 'Этаж' in label:
                result['characteristics']['floor'] = value
                if 'из' in value:
                    current, total = value.split('из')
                    result['characteristics']['floor_current'] = current.strip()
                    result['characteristics']['floor_total'] = total.strip()
            elif 'Год сдачи' in label:
                result['characteristics']['completion_year'] = value
            elif 'Дом' in label:
                result['characteristics']['building_status'] = value
            elif 'Отделка' in label:
                result['characteristics']['renovation'] = value
            else:
                result['characteristics'][label.lower()] = value

    # Местоположение
    address_elements = soup.find_all('a', {'data-name': 'AddressItem'})
    if address_elements:
        address_parts = [elem.get_text(strip=True) for elem in address_elements]
        result['location']['full_address'] = ', '.join(address_parts)

        result['location']['components'] = {
            'city': address_parts[0] if len(address_parts) > 0 else None,
            'district': address_parts[1] if len(address_parts) > 1 else None,
            'area': address_parts[2] if len(address_parts) > 2 else None,
            'street': address_parts[3] if len(address_parts) > 3 else None,
            'building': address_parts[4] if len(address_parts) > 4 else None
        }

    # Метро
    metro_items = soup.find_all('li', {'data-name': 'UndergroundItem'})
    metro_list = []

    for metro in metro_items:
        name_elem = metro.find('a', class_='xa15a2ab7--d9f62d--underground_link')
        time_elem = metro.find('span', class_='xa15a2ab7--d9f62d--underground_time')

        if name_elem:
            metro_info = {
                'name': name_elem.get_text(strip=True),
                'time': time_elem.get_text(strip=True) if time_elem else None,
                'line_color': None
            }

            metro_list.append(metro_info)

    result['location']['metro_stations'] = metro_list

    # Просмотры
    views_elem = soup.find('button', {'data-name': 'OfferStats'})
    if views_elem:
        result['metadata']['views'] = views_elem.get_text(strip=True)

    # Букинг
    booking_elem = soup.find('span', {'data-name': 'OfferBookingLabelContainer'})
    if booking_elem:
        result['additional']['online_booking'] = True

    # Инфа о девелопере
    description_span = soup.find('span', class_=lambda x: x and 'color_text-primary-default' in x and 'lineHeight_6u' in x)
    if description_span:
        developer_text = description_span.get_text(strip=True)
        if 'ELEMENT создает премиальные проекты' in developer_text or 'Element' in developer_text:
            result['developer_info']['description'] = developer_text

    # Подробная инфа
    info_groups = soup.find_all('div', {'data-name': 'OfferSummaryInfoGroup'})

    for group in info_groups:
        header = group.find('h2')
        if header:
            group_type = header.get_text(strip=True)

            items = group.find_all('div', {'data-name': 'OfferSummaryInfoItem'})

            for item in items:
                label_elem = item.find('p', class_=lambda x: x and 'color_gray60_100' in x)
                value_elem = item.find('p', class_=lambda x: x and 'color_text-primary-default' in x)

                if label_elem and value_elem:
                    label = label_elem.get_text(strip=True)
                    value = value_elem.get_text(strip=True)

                    if 'О квартире' in group_type:
                        result['characteristics'][f'flat_{label.lower().replace(" ", "_")}'] = value
                    elif 'О доме' in group_type:
                        result['characteristics'][f'building_{label.lower().replace(" ", "_")}'] = value

    # Инфа о жилом комплексе
    complex_title = soup.find('h2', string=lambda t: t and 'ЖК' in t)
    if complex_title:
        result['housing_complex']['name'] = complex_title.get_text(strip=True)

    spec_items = soup.find_all('li', class_='xa15a2ab7--_37297--item')
    for item in spec_items:
        title_elem = item.find('span', class_=lambda x: x and 'color_gray60_100' in x)
        if title_elem:
            title = title_elem.get_text(strip=True)

            value_elem = item.find('span', class_=lambda x: x and 'color_text-primary-default' in x)
            value_link = item.find('a', {'data-name': 'Link'})

            if value_elem:
                value = value_elem.get_text(strip=True)
                result['housing_complex'][title.lower().replace(" ", "_")] = value
            elif value_link:
                value = value_link.get_text(strip=True)
                result['housing_complex'][title.lower().replace(" ", "_")] = value

    # Варианты отделки
    decoration_items = soup.find_all('div', {'data-name': 'NewbuildingDecorationItem'})
    decorations = []
    for item in decoration_items:
        title_elem = item.find('span', class_=lambda x: x and 'color_text-primary-default' in x and 'fontWeight_bold' in x)
        if title_elem:
            decorations.append(title_elem.get_text(strip=True))

    if decorations:
        result['housing_complex']['decoration_options'] = decorations

    # Преимущества
    advantage_items = soup.find_all('div', {'data-name': 'AdvantageItem'})
    advantages = []
    for item in advantage_items:
        title_elem = item.find('h3')
        if title_elem:
            advantages.append(title_elem.get_text(strip=True))

    if advantages:
        result['housing_complex']['advantages'] = advantages

    return result

def create_complete_dataframe(data):
    #Создает полный DataFrame из извлеченных данных
    flat_data = {}

    # Базовые данные из HTML
    flat_data['title'] = data.get('basic_info', {}).get('title', '')
    flat_data['residential_complex'] = data.get('basic_info', {}).get('residential_complex', '')

    # Цена
    offer_data = data.get('offer_data', {})
    if 'price' in offer_data:
        price = offer_data['price']
        flat_data['price_amount'] = price.get('amount', '')
        flat_data['price_currency'] = price.get('currency', '')
        flat_data['price_type'] = price.get('price_type', '')

    # Характеристики из конфигурации
    if 'characteristics' in offer_data:
        chars = offer_data['characteristics']
        flat_data['flat_type'] = chars.get('flat_type', '')
        flat_data['total_area_config'] = chars.get('total_area', '')
        flat_data['living_area_config'] = chars.get('living_area', '')
        flat_data['decoration'] = chars.get('decoration', '')
        flat_data['floor_number'] = chars.get('floor_number', '')
        flat_data['is_apartments'] = chars.get('is_apartments', '')

    # Характеристики из HTML
    characteristics = data.get('characteristics', {})
    for key, value in characteristics.items():
        if key != 'floor':  # Пропускаем столбец floor
            flat_data[f'char_{key}'] = value

    # Локация
    location = data.get('location', {})

    # Компоненты адреса
    if 'components' in location:
        comp = location['components']
        flat_data['district'] = comp.get('district', '')
        flat_data['building'] = comp.get('building', '')

    # Метро
    metro_stations = location.get('metro_stations', [])
    if metro_stations:
        flat_data['metro_count'] = len(metro_stations)
        flat_data['metro_names'] = ', '.join([m.get('name', '') for m in metro_stations])
        flat_data['metro_times'] = ', '.join([m.get('time', '') for m in metro_stations if m.get('time')])

    # Информация о ЖК
    housing_complex = data.get('housing_complex', {})
    flat_data['hc_developer'] = housing_complex.get('застройщик', '')
    flat_data['hc_class'] = housing_complex.get('класс', '')
    flat_data['hc_building_type'] = housing_complex.get('тип_дома', '')
    flat_data['hc_parking'] = housing_complex.get('парковка', '')
    flat_data['hc_complex_type'] = housing_complex.get('тип_комплекса', '')

    # Варианты отделки
    if 'decoration_options' in housing_complex:
        flat_data['hc_decoration_options'] = ', '.join(housing_complex['decoration_options'])

    # Преимущества
    if 'advantages' in housing_complex:
        flat_data['hc_advantages'] = ', '.join(housing_complex['advantages'])

    # Информация о застройщике
    developer_info = data.get('developer_info', {})
    flat_data['developer_description'] = developer_info.get('description', '')

    # Дополнительная информация
    additional = data.get('additional', {})
    flat_data['online_booking'] = additional.get('online_booking', False)

    # Данные здания из конфигурации
    if 'building' in offer_data:
        building = offer_data['building']
        flat_data['building_material'] = building.get('material_type', '')
        flat_data['building_floors_count'] = building.get('floors_count', '')
        flat_data['building_ceiling_height'] = building.get('ceiling_height', '')
        flat_data['building_total_area'] = building.get('total_area', '')
        flat_data['building_passenger_lifts'] = building.get('passenger_lifts_count', '')
        flat_data['building_cargo_lifts'] = building.get('cargo_lifts_count', '')
        flat_data['building_deadline'] = building.get('deadline', '')

    # Новостройка
    if 'newbuilding' in offer_data:
        nb = offer_data['newbuilding']
        flat_data['nb_name'] = nb.get('name', '')
        flat_data['nb_house'] = nb.get('house', '')
        flat_data['nb_developer'] = nb.get('developer', '')
        flat_data['nb_status'] = nb.get('status', '')

    # Метаданные: дата обновления и просмотры
    metadata = data.get('metadata', {})
    flat_data['views'] = metadata.get('views', '')

    # Создаем DataFrame из одного ряда
    df = pd.DataFrame([flat_data])

    return df

def save_data_to_csv(data, filename):
    # Создаем DataFrame
    df = create_complete_dataframe(data)
    # Сохраняем в CSV
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    return df

In [ ]:
#Изначально парсинг проводился в несколько этапов, для того, чтобы компьютер не перегрузился, поэтому есть 5 файлов parsed_cards.csv с данными
async def process_files():
  df_list = []
  i = 1
  for i in range(1, 1457):
    try:
      with open(f'file_info{i}.html', 'r', encoding='utf-8') as f:
          html_content = f.read()
      # Парсим все данные
      extracted_data = parse_cian_html(html_content)
      df = create_complete_dataframe(extracted_data)
      df_list.append(df)
      print(i, end=' ')
    except Exception as e:
      print('Ошибка')
    finally:
      i += 1

  df_full = pd.concat(df_list, ignore_index=True)
  df_full.to_csv('parsed_cards.csv', index=False, encoding='utf-8-sig')

In [ ]:
await process_files()

Здесь идет соединение всех файлов и выбор необходимых столбцов для дальнейшего анализа

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
# Открываем все файлы
df1 = pd.read_csv('parsed_cards1.csv')
df2 = pd.read_csv('parsed_cards2.csv')
df3 = pd.read_csv('parsed_cards3.csv')
df4 = pd.read_csv('parsed_cards4.csv')
df5 = pd.read_csv('parsed_cards5.csv')

In [ ]:
# Объединяем
df = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)

In [ ]:
# Переименуем
df.rename(columns={'title': 'title_2'}, inplace=True)

In [ ]:
# Открываем файл с общей инфой
data = pd.read_csv('cian_parsed.csv')

In [ ]:
# Убираем строчки, к страницам объектов которых доступ мы не получили
data.drop([354, 494, 1043], inplace=True)
data.reset_index(drop=True, inplace=True)

In [ ]:
# Объединяем
data_df = pd.concat([data, df], axis=1)

In [ ]:
# Удаляем дубликаты с одинаковыми url
data_df.drop_duplicates(subset=['page_url'], inplace=True)
data_df.reset_index(drop=True, inplace=True)

In [ ]:
# Выбираем необходимые столбцы для анализа
new_data = data_df[['price', 'price_per_m2', 'description', 'title_2', 
                   'residential_complex', 'char_total_area', 'char_living_area',
                   'char_floor_current', 'char_floor_total', 'char_building_status', 
                   'char_renovation', 'char_flat_высота_потолков', 'char_building_тип_дома', 'district', 'hc_developer',
                   'hc_class', 'hc_advantages', 'online_booking', 'views', 'char_building_количество_лифтов',
                   'metro_times', ]].copy()

In [ ]:
# Создаем итоговый датафрейм, куда войдут уже обработанные столбцы
data_brushed = pd.DataFrame()

In [ ]:
# Преобразуем признаки: убираем лишние знаки, создаем более удобные столбцы
data_brushed['price'] = new_data['price'].apply(lambda x: ''.join(x.split())[:-1])
data_brushed['price_per_m2'] = new_data['price_per_m2'].apply(lambda x: ''.join(x.split())[:-4])
data_brushed['residential_complex'] = new_data['residential_complex']
data_brushed['floor_current'] = new_data['char_floor_current']
data_brushed['floor_total'] = new_data['char_floor_total']
data_brushed['hc_class'] = new_data['hc_class']
data_brushed['description'] = new_data['description']
data_brushed['views'] = new_data['views'].apply(lambda x: x.split()[0])
data_brushed['min_metro'] = new_data['metro_times'].apply(lambda x: min([int(x.split()[0]), int(x.split()[2]), int(x.split()[4])]) if isinstance(x, str) else x)
data_brushed['online_booking'] = new_data['online_booking']
data_brushed['height'] = new_data['char_flat_высота_потолков'].apply(lambda x: '.'.join(x.split(','))[:-2] if isinstance(x, str) else x)
data_brushed['total_area'] = new_data['char_total_area'].apply(lambda x: '.'.join(x.split(','))[:-2])
data_brushed['living_area'] = new_data['char_living_area'].apply(lambda x: '.'.join(x.split(','))[:-2] if isinstance(x, str) else x)

In [ ]:
# Преобразуем столбец с просмотрами к числовым значениям
data_brushed['views'] = data_brushed['views'].apply(lambda x: 0 if x == 'нет' else x)

In [ ]:
# Делаем числовой столбец для количества комнат
conditions = [
    new_data['title_2'].str.lower().str.contains('студия', na=False),
    new_data['title_2'].str.lower().str.contains(r'1-комн.', regex=True, na=False),
    new_data['title_2'].str.lower().str.contains(r'2-комн.', regex=True, na=False),
    new_data['title_2'].str.lower().str.contains(r'3-комн.', regex=True, na=False),
    new_data['title_2'].str.lower().str.contains(r'4-комн.', regex=True, na=False),
    new_data['title_2'].str.lower().str.contains(r'5-комн.', regex=True, na=False)
]

choices = [0, 1, 2, 3, 4, 5]
data_brushed['rooms'] = np.select(conditions, choices, default=None)

In [ ]:
# Делаем столбец булевым
conditions = [
    new_data['char_building_status'].str.lower().str.contains('не сдан', na=False),
    new_data['char_building_status'].str.lower().str.contains(r'сдан', regex=True, na=False)
]

choices = [False, True]
data_brushed['available'] = np.select(conditions, choices, default=None)

In [ ]:
# Делаем столбец с отделкой булевым
conditions = [
    new_data['char_renovation'].astype(str).str.contains('Чистовая|Под ключ|С отделкой'),
    new_data['char_renovation'].astype(str).str.contains('Предчистовая|Черновая|Без отделки')
]

choices = [True, False]
data_brushed['ready'] = np.select(conditions, choices, default=None)

In [ ]:
# Делаем столбце является ли рацйон городом? или это перифирия и города-спутники
conditions = [
    new_data['district'].astype(str).str.contains('р-н Адмиралтейский|р-н Центральный|р-н Василеостровский|р-н Петроградский|р-н Выборгский|р-н Калининский|р-н Фрунзенский|р-н Красногвардейский|р-н Московский|р-н Невский')
]

choices = [True]
data_brushed['is_city'] = np.select(conditions, choices, default=False)

In [ ]:
# Делаем булев столбец популярный девелопер
conditions = [
    new_data['hc_developer'].astype(str).str.contains('ЛСР. Недвижимость-Северо-Запад|Setl Group|КВС|Холдинг «РСТИ»')
]

choices = [True]
data_brushed['popular_developer'] = np.select(conditions, choices, default=False)

In [ ]:
# Обработка столбца с лифтами, чтобы привести его к числовому типу
temp_series = new_data['char_building_количество_лифтов'].fillna('').astype(str).str.lower()
passenger_matches = temp_series.str.extract(r'(\d+)\s*пассажирск')
data_brushed['elevator_p'] = passenger_matches[0].fillna(0).astype(int)
has_passenger_no_num = temp_series.str.contains('пассажирск') & ~temp_series.str.contains(r'\d+\s*пассажирск')
data_brushed.loc[has_passenger_no_num, 'elevator_p'] = 1
cargo_matches = temp_series.str.extract(r'(\d+)\s*грузов')
data_brushed['elevator_w'] = cargo_matches[0].fillna(0).astype(int)
has_cargo_no_num = temp_series.str.contains('грузов') & ~temp_series.str.contains(r'\d+\s*грузов')
data_brushed.loc[has_cargo_no_num, 'elevator_w'] = 1
data_brushed.loc[new_data['char_building_количество_лифтов'].isna(), ['elevator_p', 'elevator_w']] = np.nan

In [ ]:
data_brushed['elevators'] = data_brushed[['elevator_p', 'elevator_w']].sum(axis=1, skipna=False)

In [ ]:
data_brushed.drop(['elevator_p', 'elevator_w'], axis=1, inplace=True)

In [ ]:
# Создаем столбец есть ли двор
conditions = [
    new_data['hc_advantages'].astype(str).str.contains('Внутренний двор')
]

choices = [True]
data_brushed['court'] = np.select(conditions, choices, default=False)

In [ ]:
# Есть ли входная группа столбец 
conditions = [
    new_data['hc_advantages'].astype(str).str.contains('Входная группа')
]

choices = [True]
data_brushed['entrance'] = np.select(conditions, choices, default=False)

In [ ]:
# Создваем столбец с наличием парковки
conditions = [
    new_data['hc_advantages'].astype(str).str.contains('Парковка')
]

choices = [True]
data_brushed['parking'] = np.select(conditions, choices, default=False)

In [ ]:
# Создаем столбец с наличием парков
conditions = [
    new_data['hc_advantages'].astype(str).str.contains('Парки')
]

choices = [True]
data_brushed['parks'] = np.select(conditions, choices, default=False)

In [ ]:
# Обновляем типы
data_brushed_optimized = data_brushed.convert_dtypes()

In [ ]:
data_brushed_optimized['price'] = data_brushed_optimized['price'].astype('int')

In [ ]:
data_brushed_optimized['price_per_m2'] = data_brushed_optimized['price_per_m2'].astype('int')
data_brushed_optimized['height'] = data_brushed_optimized['height'].astype('float')
data_brushed_optimized['total_area'] = data_brushed_optimized['total_area'].astype('float')
data_brushed_optimized['living_area'] = data_brushed_optimized['living_area'].astype('float')
data_brushed_optimized['views'] = data_brushed_optimized['views'].astype('float')

In [ ]:
data_brushed_optimized.dtypes